# Implementación de un RAG de Recetas de Cocteles

## 0. Setup Inicial

In [2]:
# Instalacion de paquetes de Hugging Face
!pip install -U --quiet "transformers>=4.44.0" "accelerate>=0.33.0" "bitsandbytes>=0.43.1" "peft>=0.12.0" sentencepiece
!pip install -U --quiet --no-cache-dir bitsandbytes

import os, sys, time
print("Reiniciando el runtime para activar bitsandbytes…")
#time.sleep(1)
#os.kill(os.getpid(), 9)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 84.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 11.8 MB/s eta 0:00:00
Reiniciando el runtime para activar bitsandbytes…


In [3]:
# Stack para construir RAG
!pip -q install langchain langchain-community langchain-text-splitters faiss-cpu sentence-transformers ragas unidecode


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.7/366.7 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.9/160.9 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 74.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.8/358.8 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.4/226.

In [4]:
# Otras importaciones
import pandas as pd
import re
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
import torch, numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from textwrap import shorten

# 1. Importación de base de datos


In [5]:
# Tomamos la base de Hugging Face
cocktails = pd.read_parquet("hf://datasets/brianarbuckle/cocktail_recipes/data/train-00000-of-00001-dcec52ec7fe8275d.parquet")
columns = ['title', 'ingredients', 'directions', 'misc', 'source', 'ner']

cocktails = cocktails[(~cocktails['ingredients'].isna()) & (~cocktails['directions'].isna())][columns].reset_index(drop=True)
cocktails

,title,ingredients,directions,misc,source,ner
0,151 Swizzle,[1.5 oz. 151-Proof Demerara Rum [Lemon Hart or...,[],[],Beachbum Berry Remixed,"[pernod, rum]"
1,20th Century,"[The 21st Century, 2 oz. Siete Leguas Blanco T...","[shake on ice and strain into coupe , The Best...",[],Jim Meehan,"[cocchi americano, pernod, tequila]"
2,20th Century,"[1.5 oz. Plymouth Gin, 3\/4 oz. Mari Brizard W...",[shake on ice and strain],[],PDT,"[lillet, gin]"
3,Abbey Cocktail,[],"[Shake liquid ingredients with ice., Strain in...",[Suggested glassware is Cocktail Glass],The Ultimate Bar Book,[]
4,Absinthe Drip,[1 1/2 ounces Pernod (or other absinthe substi...,[Pour Pernod into a pousse-caf or sour glass....,[The Absinthe Drip was made famous by Toulouse...,The Ultimate Bar Book,"[pernod, absinthe]"
...,...,...,...,...,...,...
870,Yellow Bird,"[ A Caribbean favorite., 1 ounce dark rum, 1 o...","[Shake liquid ingredients with ice., Strain in...",[Suggested glassware is Cocktail Glass],The Ultimate Bar Book,"[galliano, triple sec, rum, cointreau]"
871,Yellow Fever,"[1 1/2 ounces vodka, 1/2 ounce Galliano, 1/2 o...","[Shake ingredients with ice., Strain into a ch...",[Suggested glassware is Cocktail Glass],The Ultimate Bar Book,"[vodka, galliano]"
872,Yellow Parrot Cocktail,"[3/4 ounce yellow Chartreuse, 3/4 ounce Pernod...","[Shake ingredients with ice., Strain into a ch...",[Suggested glassware is Cocktail Glass],The Ultimate Bar Book,"[brandy, pernod, chartreuse]"
873,[The Spirit of the] Algonquin,"[ 2oz. 90 Proof Rye, .75oz. Lemon Juice, .75oz...",[shake on ice and strain],[Suggested glassware is Cocktail Glass],PDT,[]


Verificamos la longitud de las recetas (la columna más larga) para determinar si debemos hacer chunking. Vemos que la longitud promedio es de 15.12 palabras y la máxima es de 94 palabras, lo cual muestra que nuestros documentos son cortos. Por lo tanto, no es necesario hacer chunks, pues seran pocos tokens.

In [6]:
# Longitud promedio
average_recipe_length_words = cocktails['directions'].apply(lambda x: len(' '.join(x).split())).mean()
print(f"La longitud promedio de la receta (número de palabras) es: {average_recipe_length_words:.2f}")

# Longitud máxima
max_recipe_length_words = cocktails['directions'].apply(lambda x: len(' '.join(x).split())).max()
print(f"La longitud máxima de la receta (número de palabras) es: {max_recipe_length_words}")

La longitud promedio de la receta (número de palabras) es: 15.12
La longitud máxima de la receta (número de palabras) es: 94


# 2. Normalización de las columnas (listas/arrays → texto)

In [7]:
def list_to_str(x):
    if isinstance(x, (list, tuple, np.ndarray)):
        return ", ".join(str(i) for i in x if str(i).strip())

    if x is None:
        return ""
    try:
        if pd.isna(x):
            return ""
    except:
        pass
    return str(x)

In [8]:
cocktails["ingredients_str"] = cocktails["ingredients"].apply(list_to_str)
cocktails["directions_str"]  = cocktails["directions"].apply(list_to_str)
cocktails["misc_str"]        = cocktails["misc"].apply(list_to_str)
cocktails["ner_str"]         = cocktails["ner"].apply(list_to_str)

# 3. Construcción del texto unificado por cóctel

El encoder necesita un solo texto por documento. Aquí combinamos:
- título,
- ingredientes,
- instrucciones,
- notas (`misc`),
- ingredientes clave (`ner`),
- y la fuente;

en una sola columna llamada `text`, que será la entrada al modelo de embeddings.

In [9]:
def row_to_text(row):
    return (
        f"Title: {row['title']}\n"
        f"Ingredients: {row['ingredients_str']}\n"
        f"Directions: {row['directions_str']}\n"
        f"Misc: {row['misc_str']}\n"
        f"Key ingredients: {row['ner_str']}\n"
        f"Source: {row['source']}"
    )

cocktails["text"] = cocktails.apply(row_to_text, axis=1)

cocktails[["title", "text"]].head()

,title,text
0,151 Swizzle,Title: 151 Swizzle\nIngredients: 1.5 oz. 151-P...
1,20th Century,Title: 20th Century\nIngredients: The 21st Cen...
2,20th Century,Title: 20th Century\nIngredients: 1.5 oz. Plym...
3,Abbey Cocktail,Title: Abbey Cocktail\nIngredients: \nDirectio...
4,Absinthe Drip,Title: Absinthe Drip\nIngredients: 1 1/2 ounce...


# 4. Encoder elegido: `sentence-transformers/all-MiniLM-L6-v2`

Usamos un **Sentence Transformer** open source (`all-MiniLM-L6-v2`) como encoder de texto a vector.

Características técnicas importantes:
- Está basado en **MiniLM**, un Transformer pequeño y eficiente (6 capas).
- Soporta **hasta 256 tokens** de entrada por texto.
- Produce un **embedding de 384 dimensiones** por cada texto completo.
- Usa *mean pooling*: promedia los embeddings de todos los tokens para obtener un solo vector por cóctel.
- Está publicado bajo licencia **Apache 2.0**, lo que permite su uso en proyectos académicos y aplicaciones prácticas.

Este encoder convierte cada receta de cóctel en un vector numérico que captura su significado.
Luego estos vectores se usan para búsqueda semántica con FAISS dentro del RAG.

In [10]:
#Encoder Elegido
embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"

#Import del Encoder
encoder = HuggingFaceEmbeddings(
    model_name=embedding_model_name
)

# Textos y metadata
texts = cocktails["text"].tolist()
metadatas = cocktails[["title", "source"]].to_dict(orient="records")

# Vectorstore FAISS (parte R de RAG)
vectorstore = FAISS.from_texts(
    texts=texts,
    embedding=encoder,
    metadatas=metadatas
)

vectorstore

/tmp/ipython-input-244656405.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  encoder = HuggingFaceEmbeddings(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
# Análisis Dimensional
lista = cocktails["text"].tolist()
cadena_mas_larga = mas_larga = max(lista, key=len)
print("La cadena más larga es:")
print(cadena_mas_larga)

print("\nEmbedding Cadena Más Larga:\n", encoder.embed_query(cadena_mas_larga))
print(f"\nDimensiones del Embedding: (1x{len(encoder.embed_query(cadena_mas_larga))})")
print(f"Tamaño Matriz de Embeddings: ({vectorstore.index.ntotal}x{len(encoder.embed_query(cadena_mas_larga))})")

La cadena más larga es:
Title: Mint Julep
Ingredients: A KENTUCKY TRADITION THAT HAS BEEN AROUND since the late 1800s, the refreshing and quintessentially Southern frosted julep is traditionally enjoyed as a celebratory libation to usher in the spring on May Day and, more importantly, is the official signature drink at the Kentucky Derby on the first Saturday in May. Traced back to the 15th century, Juleps were originally medicinal concoctions blending herbs, sugar, and water. They later evolved into drinks combining mint with brandy, peach brandy, or Cognac, until around 1875, when fine Kentucky bourbon became the spirit of choice., Traditionally, the mint’s only function was as an aromatic flourish at the edge of the drink, but inevitably, as variations ensued, controversy arose over the correct methodology, which Southerners still debate today. Some say that the key is in the placement of the mint, which should be extended above the rim to allow one to inhale the aroma while sipping

# 6. Búsqueda semántica de cócteles (parte R de RAG)

En esta parte hacemos una consulta de ejemplo. El flujo es:

1. El usuario escribe un texto (`query`) describiendo lo que quiere.
2. El encoder convierte `query` en un embedding.
3. FAISS compara ese embedding con todos los cócteles y devuelve los `k` más similares.
4. Mostramos el título, la fuente y un fragmento del texto de cada resultado.

Luego, estos `docs` serían el **contexto** que se le pasa a un modelo generativo
para construir la respuesta final (parte G de RAG).


In [12]:
query = "sweet cocktail with rum and pineapple"
docs = vectorstore.similarity_search(query, k=3)
for i, d in enumerate(docs, start=1):
    print(f"\n=== Result {i} ===")
    print("Title:", d.metadata.get("title"))
    print("Source:", d.metadata.get("source"))
    print("Snippet:\n", d.page_content[:400], "...")

def encoder(query: str):
  docs = vectorstore.similarity_search(query, k=3)
  context_text = "\n\n---\n\n".join([doc.page_content for doc in docs])
  return context_text
print(encoder(query))


=== Result 1 ===
Title: Pineapple Express
Source: New York Times
Snippet:
 Title: Pineapple Express
Ingredients: 1.5 oz. Plantation "StigGin's Fancy" Pineapple Rum, 0.75 oz. 100 proof Rhum Agricole (preferably Neisson), 0.75 oz. Lime Juice, 0.5 oz. Simple Syrup
Directions: ---shake with ice, strain into coupe glass---, Garnish with squeeze grapeFruit twist over drink and discard
Misc: 
Key ingredients: rum
Source: New York Times ...

=== Result 2 ===
Title: Spiced Rum Coco Martini
Source: The Ultimate Bar Book
Snippet:
 Title: Spiced Rum Coco Martini
Ingredients: 2 ounces spiced rum (such as Captain Morgan), 1/2 ounce chocolate liqueur (such as Godiva), Orange twist, Maraschino cherry
Directions: Stir liquid ingredients in mixing glass with ice., Strain into a chilled cocktail glass., Run the orange peel around the rim, twist it over the drink, and drop it in, along with the cherry.
Misc: Suggested glassware is C ...

=== Result 3 ===
Title: Rum Bay Breeze
Source: The Ultimate Bar Boo

# 7. Implementación del decoder

In [13]:
model_name2 = "mistralai/Mistral-7B-Instruct-v0.2"
tokenizer2 = AutoTokenizer.from_pretrained(model_name2)
model2 = AutoModelForCausalLM.from_pretrained(
      model_name2,
      torch_dtype=torch.float16,
      device_map="auto"
  )

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [14]:
tokenizer2.pad_token = tokenizer2.eos_token
model2.config.pad_token_id = model2.config.eos_token_id

from transformers import GenerationConfig

def decoder(context, query):
    prompt = f"Contexto:\n{context}\n\nPregunta: {query}\n\nRespuesta:"

    inputs = tokenizer2(prompt, return_tensors="pt").to(model2.device)

    outputs = model2.generate(
        **inputs,
        max_new_tokens=256,
        do_sample=True,
        temperature=0.7,
        top_p=0.95
    )

    full_output = tokenizer2.decode(outputs[0], skip_special_tokens=True)

    if "Respuesta:" in full_output:
        answer = full_output.split("Respuesta:")[1].strip()
    else:
        answer = full_output

    return answer

#8. Chat con el Modelo

In [16]:
try:
    while True:
        query = input("Haz una pregunta: ")
        if query.lower() in ["exit", "quit", "salir"]:
            print("Saliendo…")
            break

        context = encoder(query)
        print(decoder(context, query))

except KeyboardInterrupt:
    print("\nEjecución detenida por el usuario.")

Ask anything: Dime un coctel con Mezcal con sabor ahumado.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Spanish Fly es un coctel que contiene mezcal y Grand Marnier, y se gana su sabor ahumado al servirlo en un vaso antiguo lleno de hielo y espolvoreando café instantáneo por encima. Este coctel tiene un sabor distintivo debido a la combinación del mezcal con el sabor dulce y aromático del Grand Marnier y el café instantáneo.

Otra opción podría ser el coctel Oaxaca Chakas, que contiene mezcal, leche pesada, leche, y polvo de chocolate de Oaxaca. Este coctel se calienta en un sartén y se mezcla para crear un delicioso sabor ahumado. Además, se puede decorar con crema montada.

Si estás buscando un coctel con un sabor ahumado más fuerte, puedes considerar el coctel 20th Century, que contiene tequila blanca, Crème de Cacao, y limón, y se sir

Ejecución detenida por el usuario.
